# カテゴリデータの処理

本節ではpandasを利用したカテゴリデータの処理方法を学びます。

データ（変数）は質的変数（カテゴリ変数）、量的変数（連続変数）に分類されます。

量的変数は数や量で測れる変数で、間隔尺度と比例尺度に分類されます。

間隔尺度
: 気温や順位など、和や差に意味があるが比率には意味がない尺度（0の意味がない）

比例尺度
: 身長や重さなど、和や差、比率に意味がある尺度

質的変数は離散的な値をとり、数値では表現できない変数で、名義尺度と順序尺度に分類されます。

名義尺度
: 血液型や性別など、和や差、比率に意味がなく、比較ができない尺度

順序尺度
: 震度や企業格付など、比較できるが、和や差、比率に意味がない尺度

In [1]:
import pandas as pd
import seaborn as sns

tips = sns.load_dataset("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Category型

`tips` オブジェクトの「sex」列、「smoker」列は名義尺度のため、カテゴリ変数として扱います。Seriesではカテゴリ変数を扱うCategory型が利用できます。

In [2]:
sex = tips.loc[:, "sex"].astype("category")
sex.dtype

CategoricalDtype(categories=['Male', 'Female'], ordered=False)

In [3]:
smoker = tips.loc[:, "smoker"].astype("category")
smoker.dtype

CategoricalDtype(categories=['Yes', 'No'], ordered=False)

`tips` オブジェクトの「day」列は名義尺度のため、カテゴリ変数として扱います。Category型の生成時に順序を付けられます。

In [4]:
day_of_week = pd.CategoricalDtype(
    categories=["Sun", "Mon", "Tue", "Wed", "Thur", "Fri", "Sat"],
    ordered=True
)
day = tips.loc[:, "day"].astype(day_of_week)
day

0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
       ... 
239     Sat
240     Sat
241     Sat
242     Sat
243    Thur
Name: day, Length: 244, dtype: category
Categories (7, object): ['Sun' < 'Mon' < 'Tue' < 'Wed' < 'Thur' < 'Fri' < 'Sat']

順序が付いたCategory型のSeriesは `sort_value` メソッドによるソートが行えます。

In [5]:
day.sort_values()

0      Sun
172    Sun
173    Sun
174    Sun
175    Sun
      ... 
64     Sat
65     Sat
66     Sat
56     Sat
57     Sat
Name: day, Length: 244, dtype: category
Categories (7, object): ['Sun' < 'Mon' < 'Tue' < 'Wed' < 'Thur' < 'Fri' < 'Sat']

## カテゴリデータの変数変換

機械学習などのモデルによっては、質的変数を数値に変換する必要があります。 `get_dummies` 関数では文字列やカテゴリデータを数値に変換します。

In [6]:
pd.get_dummies(smoker)

,Yes,No
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
239,0,1
240,1,0
241,1,0
242,0,1


In [7]:
pd.get_dummies(day)

,Sun,Mon,Tue,Wed,Thur,Fri,Sat
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
239,0,0,0,0,0,0,1
240,0,0,0,0,0,0,1
241,0,0,0,0,0,0,1
242,0,0,0,0,0,0,1


In [8]:
pd.get_dummies(tips.loc[:, "time"])

,Lunch,Dinner
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
239,0,1
240,0,1
241,0,1
242,0,1


In [9]:
pd.get_dummies(tips.loc[:, ["sex", "smoker"]])

,sex_Male,sex_Female,smoker_Yes,smoker_No
0,0,1,0,1
1,1,0,0,1
2,1,0,0,1
3,1,0,0,1
4,0,1,0,1
...,...,...,...,...
239,1,0,0,1
240,0,1,1,0
241,1,0,1,0
242,1,0,0,1
